In [ ]:
import cufflinks as cf
from ipywidgets import interact

@interact
def scatter_plot(x=list(df.select_dtypes('number').columns), 
                 y=list(df.select_dtypes('number').columns)[1:],
                 theme=list(cf.themes.THEMES.keys()), 
                 colorscale=list(cf.colors._scales_names.keys())):
    
    df.iplot(kind='scatter', x=x, y=y, mode='markers', 
             xTitle=x.title(), yTitle=y.title(), 
             text='title',
             title=f'{y.title()} vs {x.title()}',
            theme=theme, colorscale=colorscale)

In [3]:
### https://github.com/jupyter-widgets/ipywidgets/blob/master/docs/source/examples/Variable%20Inspector.ipynb
import ipywidgets as widgets # Loads the Widget framework.
from IPython.core.magics.namespace import NamespaceMagics # Used to query namespace.

# For this example, hide these names, just to avoid polluting the namespace further
get_ipython().user_ns_hidden['widgets'] = widgets
get_ipython().user_ns_hidden['NamespaceMagics'] = NamespaceMagics
class VariableInspectorWindow(object):
    instance = None
    
    def __init__(self, ipython):
        """Public constructor."""
        if VariableInspectorWindow.instance is not None:
            raise Exception("""Only one instance of the Variable Inspector can exist at a 
                time.  Call close() on the active instance before creating a new instance.
                If you have lost the handle to the active instance, you can re-obtain it
                via `VariableInspectorWindow.instance`.""")
        
        VariableInspectorWindow.instance = self
        self.closed = False
        self.namespace = NamespaceMagics()
        self.namespace.shell = ipython.kernel.shell
        
        self._box = widgets.Box()
        self._box.layout.overflow_y = 'scroll'
        self._table = widgets.HTML(value = 'Not hooked')
        self._box.children = [self._table]
        
        self._ipython = ipython
        self._ipython.events.register('post_run_cell', self._fill)
        
    def close(self):
        """Close and remove hooks."""
        if not self.closed:
            self._ipython.events.unregister('post_run_cell', self._fill)
            self._box.close()
            self.closed = True
            VariableInspectorWindow.instance = None

    def _fill(self):
        """Fill self with variable information."""
        values = self.namespace.who_ls()
        self._table.value = '<div class="rendered_html jp-RenderedHTMLCommon"><table><thead><tr><th>Name</th><th>Type</th><th>Value</th></tr></thead><tr><td>' + \
            '</td></tr><tr><td>'.join(['{0}</td><td>{1}</td><td>{2}'.format(v, type(eval(v)).__name__, str(eval(v))) for v in values]) + \
            '</td></tr></table></div>'

    def _ipython_display_(self):
        """Called when display() or pyout is used to display the Variable 
        Inspector."""
        self._box._ipython_display_()
inspector = VariableInspectorWindow(get_ipython())
inspector

Box(children=(HTML(value='Not hooked'),), layout=Layout(overflow_y='scroll'))

In [ ]:
#https://towardsdatascience.com/interactive-controls-for-jupyter-notebooks-f5c94829aee6
import cufflinks as cf

@interact
def scatter_plot(x=list(df.select_dtypes('number').columns), 
                 y=list(df.select_dtypes('number').columns)[1:],
                 theme=list(cf.themes.THEMES.keys()), 
                 colorscale=list(cf.colors._scales_names.keys())):
    
    df.iplot(kind='scatter', x=x, y=y, mode='markers', 
             xTitle=x.title(), yTitle=y.title(), 
             text='title',
             title=f'{y.title()} vs {x.title()}',
            theme=theme, colorscale=colorscale)

# TEST CODE

In [ ]:
## FIGURING OUT HOW TO DISPLAY EQN AND TABLE SIDE BY SIDE
res_html = res_df.to_html()
from pprint import pprint
eqn=" $$U = \\sqrt{\\frac{ \\sum_{t=1 }^{n-1}\\left(\\frac{\\bar{Y}_{t+1} - Y_{t+1}}{Y_t}\\right)^2}{\\sum_{t=1 }^{n-1}\\left(\\frac{Y_{t+1} - Y_{t}}{Y_t}\\right)^2}}$$"

# url="['Explanation'](https://docs.oracle.com/cd/E57185_01/CBREG/ch06s02s03s04.html)"
markdown_explanation ="|Thiel's U Value | Interpretation |\n\
| --- | --- |\n\
| <1 | Forecasting is better than guessing| \n\
| 1 | Forecasting is about as good as guessing| \n\
|>1 | Forecasting is worse than guessing| \n"


pprint(res_html)
html_thiels = f"<body>\n<p>{eqn}</p>

</body>
</html>

In [ ]:
# from bs_ds.imports import *
# from bs_ds.glassboxes import *
# from test_imports import module_menu

# module_menu()

In [ ]:
# from test_imports import autoFill #as af

# from ipywidgets import HTML, HBox

# def open(value):
#     show.value=value.new
# options = [x for x in dir() if '__' not in x]    
# # options = ['Football', 'Basketball', 'Voleyball', 'Basketcase', 'Showcase', 'Footer', 'Header', 'Viewer', 'Footage', 'Showtime', 'Show Business']

# autofill = autoFill(options,callback=open)

# show = HTML('Result will be displayed here!')

# display(HBox([autofill,show]))

In [ ]:
# import ipywidgets as widgets
# from IPython.display import display

# geo={'USA':['CHI','NYC'],'Russia':['MOW','LED']}
# geoWs = {key: widgets.Select(options=geo[key]) for key in geo}

# def get_current_state():
#     return {'country': i.children[0].value,
#             'city': i.children[1].value}

# def print_city(**func_kwargs):
#     print('func_kwargs', func_kwargs)
#     print('i.kwargs', i.kwargs)
#     print('get_current_state', get_current_state())

# def select_country(country):
#     new_i = widgets.interactive(print_city, country=countryW, city=geoWs[country['new']])
#     i.children = new_i.children

# countryW = widgets.Select(options=list(geo.keys()))
# init = countryW.value
# cityW = geoWs[init]

# countryW.observe(select_country, 'value')

# i = widgets.interactive(print_city, country=countryW, city=cityW)

# display(i)

In [ ]:
# import test_imports as ti
# from test_imports import *

#import_packages
# additional_packages =  [('keras.models','models','Keras models'),
#                     ('keras.layers','layers','Keras Layers')]
# ti.import_packages(import_list_of_tuples=additional_packages)
# ti.import_packages()
# print(dir())

# Model saving/loading/plotting
https://machinelearningmastery.com/save-load-keras-deep-learning-models/

```python
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")
 
# later...
 
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
score = loaded_model.evaluate(X, Y, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))
```

In [1]:
from keras.models import model_from_json
json_file = open('NLPmodel.json')
loaded_model_json = json_file.read()
json_file.close()

loaded_model = model_from_json(loaded_model_json )

loaded_model.load_weights('NLPmodel.h5')

loaded_model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['acc'])
loaded_model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 2169, 300)         112200    
_________________________________________________________________
lstm_1 (LSTM)                (None, 200)               400800    
_________________________________________________________________
dropout_1 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                2010      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 11        
Total params: 515,021
Trainable params: 402,821
Non-trainable params: 112,200
_________

In [11]:
loaded_model.weights

[<tf.Variable 'lstm_1/kernel:0' shape=(300, 800) dtype=float32_ref>,
 <tf.Variable 'lstm_1/recurrent_kernel:0' shape=(200, 800) dtype=float32_ref>,
 <tf.Variable 'lstm_1/bias:0' shape=(800,) dtype=float32_ref>,
 <tf.Variable 'dense_1/kernel:0' shape=(200, 10) dtype=float32_ref>,
 <tf.Variable 'dense_1/bias:0' shape=(10,) dtype=float32_ref>,
 <tf.Variable 'dense_2/kernel:0' shape=(10, 1) dtype=float32_ref>,
 <tf.Variable 'dense_2/bias:0' shape=(1,) dtype=float32_ref>,
 <tf.Variable 'embedding_1/embeddings:0' shape=(374, 300) dtype=float32_ref>]

In [10]:
from keras.utils import plot_model
plot_model(loaded_model,show_shapes=True, rankdir='TB',to_file='nlp_model.png')

In [7]:
# from IPython.display import SVG
# from keras.utils import model_to_dot

# SVG(model_to_dot(loaded_model).create(prog='dot', format='svg'))

In [1]:
## NEW EVAL METHOD USING PREDICTIONS AS PART OF SLIDING WINDOW
    # # GETTING THE FIRST EVAL PATCH AND RESHAPING TO MATCH INPUT
    # first_eval_batch = train_data[-n_input:]
    # current_batch = first_eval_batch.reshape((1, n_input, n_features))
    # test_predictions=[]
    # # LOOP THROUGH ALL TEST DATA
    # for i in range(len(test_data)):

    #     # get prediction 1 time stamp ahead ([0] is for grabbing just the number instead of [array])
    #     current_pred = model.predict(current_batch)[0] 

    #     # store prediction
    #     test_predictions.append(current_pred) 

    #     # UPDATE BATCH TO INCLUDE CURRENT PREDICTION AND DROP OLDEST VALUE FROM ARRAY
    #     current_batch = np.append(current_batch[:,1:,:],[[current_pred]],axis=1) # axis===ndim

CODE THAT IS BEING REPLACE BY TIMESERIESGENERATOR
```python
# BIN STOCK_DF BY INTO X_WINDOW # TIME SEQUENCES
# Create binned columns with x_window # of sequential values, label column with next value in sequence
stock_df_binned = make_df_timeseries_bins_by_column(stock_df, x_window=x_window, verbose=0)#.iloc[:,11:])
# display(stock_df_binned.head(2).style.set_caption('Binned for Neural Network'))


# Split the training and test data by number of days
df_train_bins, df_test_bins = train_test_split_by_last_days(stock_df_binned,periods_per_day=periods_per_day,
                                                            num_test_days=num_test_days,num_train_days=num_train_days,
                                                            plot=True)
```


- And then no need to create X_train, y_train, etc

```python
# Creating X,y and index for training data and test data
train_index =df_train_bins.index
test_index = df_test_bins.index

X_train = df_train_bins['price_bins'].values
y_train = df_train_bins['price_labels'].values

X_test = df_test_bins['price_bins'].values
y_test = df_test_bins['price_labels'].values


# Converting all X and y into 2D arrays
X_train_stack =  np.vstack(X_train)
X_train_in = np.reshape(X_train_stack,(X_train_stack.shape[0],X_train_stack.shape[1],1))

X_test_stack =  np.vstack(X_test)
X_test_in = np.reshape(X_test_stack,(X_test_stack.shape[0],X_test_stack.shape[1],1))


# Set True to display array details
show_array_details = False

# Display details of array data, if desired
if show_array_details==True:
    var_dict = {'X_train':X_train_in, 'y_train':y_train,'X_test':X_test_in,'y_test':y_test}
    for name, arr in var_dict.items():
        print_array_inf(arr,name)
```

- Then model.fit replaced with fit_generator

```python
history = model.fit(X_train_in, y_train, epochs=3, verbose=2, validation_split=(0.1))
```

#### USING ROLLBACK - PANDAS GUIDE
- from the https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#dateoffset-objects

```python
In [148]: ts = pd.Timestamp('2018-01-06 00:00:00')

In [149]: ts.day_name()
Out[149]: 'Saturday'

# BusinessHour's valid offset dates are Monday through Friday
In [150]: offset = pd.offsets.BusinessHour(start='09:00')

# Bring the date to the closest offset date (Monday)
In [151]: offset.rollforward(ts)
Out[151]: Timestamp('2018-01-08 09:00:00')

# Date is brought to the closest offset date first and then the hour is added
In [152]: ts + offset
Out[152]: Timestamp('2018-01-08 10:00:00')
```

### add AIC and BIC

#### NOTES ON Evaluating Forecast Accuracy
- **Mean Absolute Error, Mean Squared Error, Root Mean Squared Error**
    - A forecast method that minimizes the **MAE** will lead to **forecasts of the median*
    - while minimizing the **RMSE will lead to forecasts of the mean.**
    
- **AIC / BIC**
    - Akaike Information Criterion (AIC)
    - Bayesian Information Criterion (BIC)
    
The AIC evaluates a collection of models and estimates the quality of each model relative to the others. Penalties are provided for the number of parameters used in an effort to thwart overfitting. The lower the AIC and BIC, the better the model should be at forecasting.

These functions are available as

    from from statsmodels.tools.eval_measures import aic, bic
    


## METHOD OF CONSTRUCTING DATA FROM GENERATOR / TESTING THIEL'S U

In [ ]:
# REVERTING BACK TO ORIGINAL METHOD NOW THAT I UNDERSTAND I NEED TO SHIFT TRUE VALUES -1
# plot_preds_with_thiels_u(mode, test_generator,)
# GET PREDICTIONS FROM TEST_GENERATOR
preds_from_gen = pd.Series(model.predict_generator(test_generator).ravel(), 
                             index=test_data_index[n_input:], name='preds_from_gen')

# EXTRACT TARGET DATA (TRUE VALUES USED))
true_data = [ i[1][0].tolist()[0] for i in test_generator]
true_from_gen = pd.Series(data=true_data,index=test_data_index[n_input:], name='true_from_gen') # get the target values
df_U = pd.concat([true_from_gen,preds_from_gen],axis=1)

display(df_U.head())
df_U.plot()
plt.legend()

U =thiels_U(df_U['true_from_gen'].values,df_U['preds_from_gen'].values)
plt.title('True Price vs Predicted Price')
print("Thiel's U = ",U)

In [1]:
## NEW EVAL METHOD USING PREDICTIONS AS PART OF SLIDING WINDOW
    # # GETTING THE FIRST EVAL PATCH AND RESHAPING TO MATCH INPUT
    # first_eval_batch = train_data[-n_input:]
    # current_batch = first_eval_batch.reshape((1, n_input, n_features))
    # test_predictions=[]
    # # LOOP THROUGH ALL TEST DATA
    # for i in range(len(test_data)):

    #     # get prediction 1 time stamp ahead ([0] is for grabbing just the number instead of [array])
    #     current_pred = model.predict(current_batch)[0] 

    #     # store prediction
    #     test_predictions.append(current_pred) 

    #     # UPDATE BATCH TO INCLUDE CURRENT PREDICTION AND DROP OLDEST VALUE FROM ARRAY
    #     current_batch = np.append(current_batch[:,1:,:],[[current_pred]],axis=1) # axis===ndim

### NOTES ON SHAPE OF EVAL BATCH
- Need to match the 3 dimension shape of the input from the generator

```python
first_eval_batch = scaled_train[-n_input:] # ? he had 12, and 12=his n_input
current_batch = first_eval_batch.reshape((1, n_input, n_features))
```

###  NOTES ON NUMBER FORMATTING
- https://pbpython.com/styling-pandas.html
- Reference table for formatting: https://mkaz.blog/code/python-string-format-cookbook/


```python
fmt_price = '${0:,.0f}'
format_dict = {'Predictions': '{:0.2f}',  'Test_values': '{:0.2f}', 
'True Test Price':fmt_price,'True Predictions Price':fmt_price }
df_show.style.format(format_dict).hide_index()
```

## NOTES ON FORECASTING INTO FUTURE

- Loop to get next prediction based on current eval_batch
- Append prediction to next eval batch and drop the oldest value in the current eval_batch
- To predict into the TRUE FUTURE, extend the `for i in range(len(test)):` line
    - replace range(len(test)) with however far you want to predict.
    - Data will get _much noisier_ farther out into TRUE future.

```python
test_predictions = []

# GETTING THE FIRST EVAL PATCH AND RESHAPING TO MATCH INPUT
first_eval_batch = scaled_train[-n_input:]
current_batch = first_eval_batch.reshape((1, n_input, n_features))

# LOOP THROUGH ALL TEST DATA
for i in range(len(test)):
    
    # get prediction 1 time stamp ahead ([0] is for grabbing just the number instead of [array])
    current_pred = model.predict(current_batch)[0] 
    
    # store prediction
    test_predictions.append(current_pred) 
    
    # UPDATE BATCH TO INCLUDE CURRENT PREDICTION AND DROP OLDEST VALUE FROM ARRAY
    current_batch = np.append(current_batch[:,1:,:],[[current_pred]],axis=1) # axis===ndim

print(test_predictions)

display(scaled_test)
```

In [ ]:
test_predictions = []

# SHAPE OF 
first_eval_batch = scaled_train[-n_input:]
current_batch = first_eval_batch.reshape((1, n_input, n_features))

for i in range(len(test)):
    
    # get prediction 1 time stamp ahead ([0] is for grabbing just the number instead of [array])
    current_pred = model.predict(current_batch)[0]
    
    # store prediction
    test_predictions.append(current_pred) 
    
    # update batch to now include prediction and drop first value
    current_batch = np.append(current_batch[:,1:,:],[[current_pred]],axis=1)

df_show=pd.DataFrame({'Predictions':[x[0] for x in test_predictions],'Test Values':scaled_test.ravel()})    
# print(test_predictions)
# display(scaled_test)
display(df_show)

In [ ]:
def test_eval_batches(model,scaled_train_data, scaled_test_data, n_input,n_features):
    test_predictions = []
    first_eval_batch=[]
    # take the last window size of data from training data 
    first_eval_batch = scaled_train_data[-n_input:]
    current_batch = first_eval_batch.reshape((1, n_input, n_features))
    
    for i in range(len(test_data)):
        print(i)
        # get prediction 1 time stamp ahead ([0] is for grabbing just the number instead of [array])
        current_pred = model.predict(current_batch)[0]
        
        # store prediction
        test_predictions.append(current_pred) 

        # update batch to now include prediction and drop first value
        current_batch = np.append(current_batch[:,1:,:],[[current_pred]],axis=1)
    return test_predictions

new_preds = test_eval_batches(model,train_data,test_data,n_input,n_features)

In [ ]:
# model1.predict()
scaled_train_data = df_train['price']
scaled_test_data = df_test['price']
n_input= model_params['input_params']['n_input']
n_features = model_params['input_params']['n_features']

def get_preds_from_preds(model1,scaled_train_data, scaled_test_data, model_params=None, 
                      first_eval_from_train_data=True, return_df=True,suffix=None):
    test_predictions = []
    first_eval_batch=[]
    index_tracker = []
    if model_params is not None:
#         train_data_index = model_params['input_params']['train_data_index']
#         test_data_index = model_params['input_params']['test_data_index']
        n_input =  model_params['data_params']['x_window']


    preds_out = [['i','index','pred_price']]
    
    # making series into arrays and saving index
    train_data_index = scaled_train_data.index#[-n_input:]
    scaled_train_data = scaled_train_data.values#[-n_input:].values

    # making series into arrays and saving index
    test_data_index = scaled_test_data.index
    scaled_test_data = scaled_test_data.values
    
    
    if first_eval_from_train_data:
        loop_length = range(len(scaled_test_data))
        
        # take the last window size of data from training data 
        first_eval_batch = scaled_train_data[-n_input:]
        first_batch_idx = train_data_index[-n_input:]

    else:
        loop_length = range(n_input,len(scaled_test_data))
        first_eval_batch = scaled_test_data[:n_input]
      
    
    current_batch = first_eval_batch.reshape((1, n_input, n_features))

    for i in loop_length:

        # get prediction 1 time stamp ahead ([0] is for grabbing just the number instead of [array])
        current_pred = model.predict(current_batch)[0]
        preds_out.append([i,test_data_index[i],current_pred[0]])
        
        
        # store prediction
        test_predictions.append(current_pred) 

        # update batch to now include prediction and drop first value
        current_batch = np.append(current_batch[:,1:,:],[[current_pred]],axis=1)
        
    if return_df:
        res_df = bs.list2df(preds_out)
        res_df['index'] = pd.to_datetime(res_df['index'])
        res_df.set_index('index',inplace=True)
        
        if suffix is not None:
            
            colnames = [name+suffix for name in res_df.columns]
        else:
            colnames = res_df.columns
            
        res_df.columns=colnames
        return res_df #test_predictions, checks
    
    else:
        print('reutrning nothing')
#         return test_predictions
    
    
    
def get_preds_from_gen(model,test_generator,true_test_data,model_params,return_combined=False):
        
        if model_params is not None:
            train_data_index = model_params['input_params']['train_data_index']
            test_data_index = model_params['input_params']['test_data_index']
            x_window =  model_params['data_params']['x_window']

        
        gen_index = true_test_data.index[test_generator.start_index:test_generator.end_index+1]
        gen_true_targets = test_generator.targets[test_generator.start_index:test_generator.end_index+1]
        gen_preds = model.predict_generator(test_generator)
        gen_preds = gen_preds.ravel()
        gen_true_targets = gen_true_targets.ravel()
        
        gen_pred_df = pd.DataFrame({'index':gen_index,'gen_true_target':gen_true_targets,'gen_preds':gen_preds})
        gen_pred_df['index'] = pd.to_datetime(gen_pred_df['index'])
        gen_pred_df.set_index('index',inplace=True)
        return gen_pred_df
    

    
    
    
gen_df = get_preds_from_gen(model1, test_generator,df_test['price'],model_params)    
func_df = get_preds_from_preds(model1,df_train['price'],df_test['price'],model_params,
                               suffix='_func',first_eval_from_train_data=False)+
bs.display_side_by_side(gen_df,func_df)

In [ ]:
# ##### SUPER IMPORTANT CELL
# scaled_train_data = df_train['price']
# scaled_test_data = df_test['price']

# get_model_preds_from_preds= ji.get_model_preds_from_preds
# def get_preds_from_preds(model1,scaled_train_data, scaled_test_data,
#                          model_params=None, n_input=None, n_features=None, 
#                          build_preds_from_train=True, return_df=True,suffix=None):
    
#     """ Gets predictions from model using using its own predictions as the subsequent input.
#     Must provide a model_params dictionary with 'input_params' OR must define ('n_input','n_features').
    
#     * IF build_preds_from_train is True:
#         1. starting true time series for predictions is the last rows [-n_input:] from training data.
#         2. output predicitons will be the same length as the input scaled_test_data
    
#     * IF build_preds_from_train is False:
#         1. starting true time series for predictions is the first rows [:n_input] from test data.
#         2. output predicitons will be shorter by n_input # of rows
#         """
#     test_predictions = []
#     first_eval_batch=[]
#     index_tracker = []
    
#     if model_params is not None:
#         n_input= model_params['input_params']['n_input']
#         n_features = model_params['input_params']['n_features']

#     if model_params is None:
#         if n_input is None or n_features is None:
#             raise Exception('Must provide model params or define n_input and n_features')
        

#     preds_out = [['i','index','pred']]
    
#     # SAVING COPY OF INPUT TEST DATA
#     scaled_test_series = scaled_test_data.copy() 

#     ## SAVING TRAIN AND TEST DATA INDICES AND VALUES
#     train_data_index = scaled_train_data.index
#     scaled_train_data = scaled_train_data.values
#     test_data_index = scaled_test_data.index
#     scaled_test_data = scaled_test_data.values
    
    
#     ## PREPARE THE FIRST EVAL BATCH TIMESERIES FROM TRAIN OR TEST DATA
#     # Change parameters depending on if from train or test data
#     if build_preds_from_train:
        
#         # If using trianing data loop through full test data
#         loop_length = range(len(scaled_test_data))
        
#         # take the last window size of data from training data 
#         first_eval_batch = scaled_train_data[-n_input:]
#         first_batch_idx = train_data_index[-n_input:]
        
#         # set the true index to test_data_index
#         true_index_out = test_data_index
        
#     # Set the loop to be from n_input # of rows into test_data
#     else:
#         loop_length = range(n_input,len(scaled_test_data))
#         first_eval_batch = scaled_test_data[:n_input]
#         true_index_out =  test_data_index[n_input:]
      
    
#     # reshape first batch of data for model.predict 
#     current_batch = first_eval_batch.reshape((1, n_input, n_features))

    
#     ## LOOP THROUGH REMAINING TIMEBINS USING CURRENT PREDICITONS AS NEW DATA FOR NEXT
#     for i in loop_length:

#         # get prediction 1 time stamp ahead ([0] is for grabbing just the number instead of [array])
#         current_pred = model.predict(current_batch)[0]        
#         # store prediction
#         test_predictions.append(current_pred) 

#         # update batch to now include prediction and drop first value
#         current_batch = np.append(current_batch[:,1:,:],[[current_pred]],axis=1)
        
#         ## Append the data to the output df list
#         preds_out.append([i,test_data_index[i],current_pred[0]])


#     ## If returning a dataframe,prepare and rename with suffix
#     if return_df:
#         res_df = bs.list2df(preds_out)
#         res_df['index'] = pd.to_datetime(res_df['index'])
#         res_df.set_index('index',inplace=True)
        
#         # adding true price
#         res_df['true'] = scaled_test_series.loc[true_index_out]
#         res_df=res_df[['i','true','pred']]
        
#         if suffix is not None:
#             colnames = [name+suffix for name in res_df.columns]
#         else:
#             colnames = res_df.columns
            
#         res_df.columns=colnames
#         return res_df #test_predictions, checks
    
#     # Else just return array of predictions
#     else:
#         return test_predictions

    
    
# def get_model_preds_from_gen(model,test_generator,true_test_data,model_params=None,
#                        n_input=None, n_features=None, suffix=None, return_df=False):
#         """
#         Gets prediction from model using the generator's timeseries using model.predict_generator()
#         Must provide a model_params dictionary with 'input_params' OR must define ('n_input','n_features').

#         """
#         if model_params is not None:
#             n_input= model_params['input_params']['n_input']
#             n_features = model_params['input_params']['n_features']

#         if model_params is None:
#             if n_input is None or n_features is None:
#                 raise Exception('Must provide model params or define n_input and n_features')

#         # GET TRUE VALUES AND DATETIME INDEX FROM GENERATOR
        
#         # Get true time index from the generator's start_index and end_index 
#         gen_index = true_test_data.index[test_generator.start_index:test_generator.end_index+1]
#         gen_true_targets = test_generator.targets[test_generator.start_index:test_generator.end_index+1]
        
        
#         # Generate predictions from the test_generator
#         gen_preds = model.predict_generator(test_generator)
#         gen_preds_flat = gen_preds.ravel()
        
#         gen_true_targets = gen_true_targets.ravel()
        
        
#         # RETURN OUTPUT AS DATAFRAME OR ARRAY OF PREDS
#         if return_df:
#             # Combine the outputs
#             gen_pred_df = pd.DataFrame({'index':gen_index,'true':gen_true_targets,'pred':gen_preds_flat})
#             gen_pred_df['index'] = pd.to_datetime(gen_pred_df['index'])
#             gen_pred_df.set_index('index',inplace=True)

#             if suffix is not None:
#                 colnames = [name+suffix for name in gen_pred_df.columns]
#             else:
#                 colnames = gen_pred_df.columns
#             return gen_pred_df
#         else:
#             return gen_preds


# def compare_model_pred_methods(model, true_train_series,true_test_series, test_generator=None,
#                                model_params=None, n_input=None, n_features=None, from_gen=True,
#                                from_train_series = True, from_test_series=True, 
#                                iplot=True, plot_with_train_data=True,return_df=True, inverse_tf=True):
#     """ Gets predictions for training data from the 3 options: 
#     1) from generator  --  len(output) = (len(true_test_series)-n_input)
#     2) from predictions on test data  --  len(output) = (len(true_test_series)-n_input)
#     3) from predictions on train data -- len(true_test_series)
#     """
#         if model_params is not None:
#             n_input= model_params['input_params']['n_input']
#             n_features = model_params['input_params']['n_features']

#         if model_params is None:
#             if n_input is None or n_features is None:
#                 raise Exception('Must provide model params or define n_input and n_features')
                
#         if from_gen is True and test_generator is None:
#             raise Exception('If from_gen=True, must provide generator.')

                
#         ### GET the 3 DIFERENT TYPES OF PREDICTIONS    
#         df_list = []
#         if from_gen:
#             gen_df = ji.get_model_preds_from_gen(model, test_generator,true_test_series,
#                                                  model_params=model_params, n_input=n_input, n_features=n_features, 
#                                                 suffix='_from_gen',return_df=True)    
#             df_list.append(gen_df)

#         if from_test_series:
#             func_df_from_test = ji.get_model_preds_from_preds(model, true_train_series,true_test_series,
#                                                           model_params=model_params, n_input=n_input, 
#                                                           n_features=n_features, suffix='_from_test',
#                                                           build_preds_from_train=False, return_df=True)
#             df_list.append(func_df_from_test)

#         if from_train_series:
#             func_df_from_train  = ji.get_model_preds_from_preds(model,true_train_series,true_test_series,
#                                                                 model_params=model_params, n_input=n_input,
#                                                                 suffix='_from_train', build_preds_from_train=True,
#                                                             return_df=True)
#             df_list.append(func_df_from_train)

#         # bs.display_side_by_side(func_df,func_df_from_train)

#         df_all_preds = pd.concat([df for df in df_list],axis=1)
#         df_all_preds = bs.drop_cols(df_all_preds,['i_'])


#         if inverse_tf:
#             df_out = ji.transform_cols_from_library(df_all_preds,single_scaler=model_params['scaler_library']['price'],inverse=True)
#         else:
#             df_out = df_all_preds

#         if iplot:
#             if plot_with_train_data is False:
#                 ji.plotly_time_series(df_out)
#             else:
#                 true_train_price = ji.inverse_transform_series(true_train_series,model_params['scaler_library']['price'])
#                 df_plot = pd.concat([true_train_price,df_out],axis=1)
#                 ji.plotly_time_series(df_plot)

#         if return_df:
#             return df_out

# ✔BOOKMARK: NOW MUST FIGURE OUT HOW TO GET THE EVALUATE_REGRESSION FOR THE DF_MODEL WITH MULTIPLE PREDS
## ALSO - CODE FROM TESTING GENERATOR PREDICTIOND
- ## USED NEW `get_evaluate_regression_dict()`

In [ ]:

# from keras.preprocessing.sequence import TimeseriesGenerator
# batch_size=1
# n_input = model_params['data_params']['x_window']
# print(n_input)
# # batch_size = model_params['data_params']['batch_size']
# # RESHAPING TRAINING AND TEST DATA 

# test_data_series = df_test['price']
# print(test_data_series.shape)
# test_data = test_data_series.values.reshape(-1,1)
# print(test_data.shape)
# test_data_index =test_data_series.index
# len(test_data)

In [ ]:
# true_test_data = df_test['price']
# true_train_data = df_train['price']

# test_gen_index = true_test_data.index[test_generator.start_index:test_generator.end_index+1]
# test_gen_preds = model1.predict_generator(test_generator)
# test_gen_true_targets = test_generator.targets[test_generator.start_index:test_generator.end_index+1]

# train_gen_index = true_train_data.index[train_generator.start_index:train_generator.end_index+1]
# train_gen_preds = model1.predict_generator(train_generator)
# train_gen_true_targets = train_generator.targets[train_generator.start_index:train_generator.end_index+1]


# print('---'*3,'TRAIN DATA','---'*3)
# print('gen_index: ', train_gen_index.shape)
# print('gen_preds: ', train_gen_preds.shape)
# print('gen_true_targets: ', train_gen_true_targets.shape)


# print('---'*3,'TEST DATA','---'*3)
# print('gen_index: ', test_gen_index.shape)
# print('gen_preds: ', test_gen_preds.shape)
# print('gen_true_targets: ', test_gen_true_targets.shape)


#     # Generate predictions from the test_generator
# test_gen_preds = model1.predict_generator(test_generator)
# test_gen_preds_flat = test_gen_preds.reshape() #ravel()
# test_gen_true_targets = test_gen_true_targets.ravel()

# print('---'*3,'TEST DATA RAVEL','---'*3)
# print('gen_index: ', test_gen_index.shape)
# print('gen_preds_flat: ', test_gen_preds_flat.shape)
# print('gen_preds: ', test_gen_preds.shape)
# print('gen_true_targets: ', test_gen_true_targets.shape)
# # plt.plot(test_gen_preds)

# test_gen_preds

# TWITTER TS PROCESSING

In [ ]:
## PREVIEWING EFFECT OF ROLLING OFFSETS
def test_ofst_rolling(i=0):
    """Tool to help visualize the effect of offsets when deterimining a time interval range."""
    ofst = pd.offsets.CustomBusinessHour(start='09:30',end='16:30',)
    print('TS: \t\t',twitter_df.index[i])
    print('TS.floor(H): \t',twitter_df.index[i].floor('H'))
    print('TS rollback: \t\t',ofst.rollback(twitter_df.index[i]))
    print('TS rollforward: \t',ofst.rollforward(twitter_df.index[i]))
    print('\n')
    print('TS floor(H)-rollback: \t',ofst.rollback(twitter_df.index[i].floor('H')))
    print('TS floor(H)-rollforward: ',ofst.rollforward(twitter_df.index[i].floor('H')))

test_ofst_rolling(0)

# REMOVED FOR MEET WITH BRANDON

##  ~~KERAS MODEL 2B~~
Blog Post Code
- NEW BLOG POST - LSTM TIME SERIES FORECASTING
    - https://machinelearningmastery.com/how-to-develop-lstm-models-for-time-series-forecasting/
- https://www.analyticsvidhya.com/blog/2018/10/predicting-stock-price-machine-learningnd-deep-learning-techniques-python/

```python
from keras.layers import TimeDistributed, Conv1D, MaxPool1D, Flatten, LSTM, Dense, MaxPooling1D
model3 = Sequential()
# input_shape =  (X_train_in.shape[0],X_train_in.shape[1],1)
input_shape=(n_input, n_features,1)

model3.add(TimeDistributed(Conv1D(filters=64, kernel_size=1, activation='relu'), input_shape=input_shape)) #(None, n_steps, n_features)))
model3.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model3.add(TimeDistributed(Flatten()))
model3.add(LSTM(50, activation='relu'))
model3.add(Dense(1))
model3.compile(optimizer='adam', loss=my_rmse)
model3.summary()
# fit model

clock.tic('')
history3 = model3.fit_generator( train_generator,epochs=3,verbose=2,callbacks=callbacks,workers=3)

clock.toc('')
evaluate_model_plot_history(model3, train_generator, test_generator)
```

-#
\# ✔ BOOKMARK - RESUMING ANALYSIS WITH NEW FUNCTIONS
- Check Evernote log: https://www.evernote.com/l/AApgV1EaL9lEYpujWnwHq62W9QyEKkH4hwA/
